In [ ]:
# Debian (Quarto)
!sudo curl -LO https://quarto.org/download/latest/quarto-linux-amd64.deb
!sudo apt-get install gdebi-core -y
!sudo gdebi -n quarto-linux-amd64.deb
!/usr/local/bin/quarto check

# Alternative installation using nbdev
# nbdev_install_quarto

Reading package lists... Done
Building dependency tree        
Reading state information... Done
Reading state information... Done
Selecting previously unselected package quarto.
(Reading database ... 32611 files and directories currently installed.)
Preparing to unpack quarto-linux-amd64.deb ...
Unpacking quarto (1.1.251) ...
Setting up quarto (1.1.251) ...


In [ ]:
# Debian (Java JDK)
!sudo apt update
!sudo apt install default-jdk

In [ ]:
# nbdev
!conda install -c fastai nbdev
#!pip install nbdev
#!nbdev_help

Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done


In [3]:
# Merge notebooks
import nbformat
def merge_notebooks(notebooks, output, ignore_index = True):
    # ignore_index enabled will assign new ids to cells
    merged_nb, metadata = None, []
    for path in notebooks:
        # Read the notebooks
        with open(path, 'r', encoding='utf-8') as handler:
            nb = nbformat.read(handler, as_version=4)
            metadata += [nb.metadata]
        
        if merged_nb is None:
            merged_nb = nb
        else:
            merged_nb.cells.extend(nb.cells)

        merged_metadata = {}
        for meta in reversed(metadata):
            merged_metadata.update(meta)
        merged_nb.metadata = merged_metadata

    # Create a combined notebook
    if ignore_index:
        nbformat.write(nbformat.v4.new_notebook(cells=merged_nb.cells, metadata=merged_nb.metadata), output)
    else:
        nbformat.write(merged_nb, output)

In [4]:
# Clean specific metadata (!mutate), tags etc. (nbconvert, nbformat and nbdev)
import nbformat

# Clean metadata and output
#!nbdev_clean --fname report-example.ipynb

# Remove cells with specific tags
def clear_tag(path, key):
    nb = nbformat.read(path, as_version=4)
    for i, cell in enumerate(nb["cells"]):
        try:
            cell["metadata"]["tags"][key].remove("raises-error")
            if not cell["metadata"]["tags"]:
                cell["metadata"].pop("tags")
        except:
            pass
        nb["cells"][i] = cell
    nbformat.write(nb, path)

# Remove cells with specific metadata
def remove_cell_by_metadata(path, key):
    nb = nbformat.read(path, as_version=4)

    # Repeat process as index changes on pop
    repeat = len(nb.cells)
    for i in range(repeat):
        nb = nbformat.read(path, as_version=4)
        # Find cell with metadata key, pop, repeat
        for i, cell in enumerate(nb.cells):
            cell_metadata = cell.metadata.get(key)
            if cell_metadata:
                nb.cells.pop(i)
                continue
    nbformat.write(nb, path)

# Remove empty cells
def remove_empty_cells(path):
    nb = nbformat.read(path, as_version=4)

    # Repeat process as index changes on pop
    repeat = len(nb.cells)
    for i in range(repeat):
        nb = nbformat.read(path, as_version=4)
        # Find empty cell (no source), pop, repeat
        for i, cell in enumerate(nb.cells):
            cell_source = cell.source
            if len(cell_source) == 0:
                nb.cells.pop(i)
                continue
    nbformat.write(nb, path)

# todo: combine the techniques into a more general function or alt. a class or functional approach 

In [7]:
# Prepare report for Quarto
import os
import pathlib
import shutil
import glob
def report(path, output, preview = True):
    # Read environment e.g., is it Deepnote? Clean non-compatible metadata..
    path_abs = os.path.abspath(path)
    output_abs = os.path.abspath(output)

    # Create temp build directory in output
    build_path = os.path.join(output_abs, '.report-build')
    if not os.path.exists(build_path):
        os.makedirs(build_path)

    # Read files in directory (sort the names e.g., 01-section, 02-section etc.)
    notebooks = sorted(glob.glob(os.path.join(path_abs, '*.ipynb')))

    # Create output file and replace with merge
    report_path = os.path.join(build_path, 'report.ipynb')
    merge_notebooks(notebooks, report_path)

    # Remove cells by metadata key
    remove_cell_by_metadata(report_path, 'created_in_deepnote_cell')

    # Remove empty cells
    remove_empty_cells(report_path)

    # Copy final report to output
    final_report_path = os.path.join(output_abs, 'report.ipynb')
    shutil.copy(report_path, final_report_path)

    # Preview enabled opens Quarto preview of report
    # todo: detected changes should merely update current session
    if preview:
        pass
    
    # Quarto render report (options: execute, [docx, pdf, html, etc.])
    !quarto install tool tinytex
    !quarto render $final_report_path --to pdf

    # Clear temp build directory
    if os.path.exists(build_path):
        shutil.rmtree(build_path)
    else:
        print("Error: %s directory not found" % build_path)

# todo: if any step fails clean temp files etc.
# todo: add quarto options to generated report e.g., include code, outputs etc.. 

# Test
#report(path='report-example', output='report-example/output')

In [12]:
!echo "$PS1"
!printenv


SHELL=/bin/bash
DEEPNOTE_GITHUB_TOKEN_SERVICE_URL=https://deepnote.com/api/github-apps/git-token
KUBERNETES_SERVICE_PORT_HTTPS=443
JUPYTER_CONFIG_DIR=/deepnote-config/jupyter
PIP_TARGET=/opt/conda/lib/python3.8/site-packages
KUBERNETES_SERVICE_PORT=443
INTEGRATION_CREDENTIALS_SERVICE_PORT=tcp://10.100.12.67:80
HOSTNAME=p-7393f88e-6674-4a74-8a6c-ce9dc44581ca
PYTHON_VERSION=3.8
LANGUAGE=en_US.UTF-8
GIT_SSH_COMMAND=ssh -i /work/.deepnote/gitkey -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no
INTEGRATION_CREDENTIALS_SERVICE_PORT_80_TCP_ADDR=10.100.12.67
INTEGRATION_CREDENTIALS_SERVICE_PORT_80_TCP=tcp://10.100.12.67:80
PWD=/work/apps/explorer
DEEPNOTE_PYTHON_KERNEL_ONLY=true
JUPYTER_TOKEN=b0106297-544c-4bd3-85f8-63c4e41c0b45
PYTHON_SETUPTOOLS_VERSION=57.5.0
_=/usr/bin/printenv
INTEGRATION_CREDENTIALS_SERVICE_SERVICE_PORT=80
PLOTLY_RENDERER=colab
IPYTHONDIR=/deepnote-config/ipython
HOME=/root
LANG=en_US.UTF-8
KUBERNETES_PORT_443_TCP=tcp://10.100.0.1:443
INTEGRATION_CREDENTIALS_S

In [1]:
prompt = [!echo "$PS0", !echo "$PS1", !echo "$PS2", !echo "$PS3"]


SyntaxError: invalid syntax (3477522534.py, line 1)

In [ ]:
# Gather information about current host system (from host shell won't be accurate from notebooks)
def system_information():

    # Prompt
    prompt = [!echo "$PS0", !echo "$PS1", !echo "$PS2", !echo "$PS3"]

    # User

    # System    

# Gather information about current kernel (inside notebooks)
def kernel_information():

    # Prompt
    prompt = [!echo "$PS0", !echo "$PS1", !echo "$PS2", !echo "$PS3"]

    # User

    # System    

# Explorer

@dataclass
class Explorer:

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7393f88e-6674-4a74-8a6c-ce9dc44581ca' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>